In [1]:
# Instalar o google-genai
%pip -q install google-genai

In [2]:
# Importar as dependências
import os
from google.colab import userdata
from google import genai
from google.genai import types
from google.colab import files # para poder selecionar arquivos de sua máquina e enviar para o colab
from IPython.display import display, HTML, Markdown, Image # Para exibir texto formatado no Colab
import textwrap # Para formatar melhor a saída de texto

In [3]:
# Criar variável de ambiente com sua GOOGLE_API_KEY
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [4]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [7]:
# função responsável pelo chat
def chat():

    client = genai.Client(api_key=os.environ.get("GEMINI_API_KEY"))
    model = "gemini-2.0-flash"
    generate_content_config = types.GenerateContentConfig(
    response_mime_type="text/plain",
    system_instruction = """
      Você é um assistente de jardinagem chamado JarGemini, mas também é especialistas em plantas frutiferas e ornamentais.
      Você sempre apresenta respostas que podem ajudar a resolver os problemas das plantas das pessoas,
      mas evitar dar respostas muito longas, somente dá respostas longas quando realmente é necessário.

      Você informa dados técnicos, como nomes de espécies, mas sempre usa uma linguagem simples, para
      que todas as pessoas possam entender, mesmo que não sejam especialistas como você.
      Quase sempre, quando é relevante, você adiciona links de vídeos, fotos, ou outros tipos de link,
      ao final da resposta, para torná-la mais completa.

      Caso não seja enviada uma imagem para análise ou a pergunta é não tem detalhes suficientes, ou não
      façam sentido em relação ao histórico da conversa, você pedirá mais detalhes, ou que envie uma foto,
      para que você possa entender melhor o caso e tentar ajudar da melhor forma possível.

      Você adora ajudar as pessoas com suas plantas e sempre que possível aproveita para demonstrar que
      é um grande palmeirense.
    """
    )

    # Lista para armazenar o histórico de mensagens do chat
    chat_history = []

    while True:
        print("Faça sua pergunta sobre plantas: ")
        user_input = input()

        if user_input.lower() == "sair":
            break

        filename = None
        contents = []
        uploaded_file_path = None # Variável para armazenar o caminho do arquivo local

        # Verifica se o usuário deseja enviar um arquivo
        print("-------------------------------------------------------------")
        print("Gostaria de enviar foto referente o assunto ? (Caso não deseje enviar uma foto é só cancelar): ")
        uploaded = files.upload()
        if uploaded:
          filename = list(uploaded.keys())[0]
          uploaded_file_path = filename # Armazena o caminho local do arquivo
          print("-------------------------------------------------------------")
          print("Visualização da imagem enviada:")
          display(Image(filename=uploaded_file_path)) # Exibe a imagem diretamente no Colab

        contents.append(
            types.Content(
                role="user",
                parts=[types.Part.from_text(text=user_input)],
            )
        )
        # Verifica se o input do usuário parece ser um caminho de arquivo de imagem
        if filename and os.path.exists(filename) and filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
            try:
                # Faz o upload do arquivo
                uploaded_file = client.files.upload(file=filename)
                if uploaded_file:
                    # Adiciona a imagem à lista de conteúdos
                    contents.append(
                        types.Content(
                            role="user",
                            parts=[
                                types.Part.from_uri(
                                    file_uri=uploaded_file.uri,
                                    mime_type=uploaded_file.mime_type,
                                )
                            ],
                        )
                    )
                    # uma vez que já foi feito o upload para o Gemini excluimos arquivo do Colab
                    os.system(f"rm -f {filename}")
                else:
                    print("Erro ao fazer upload do arquivo")
                    continue

            except Exception as e:
                print(f"Erro ao processar a imagem: {e}")
                continue

        # Adiciona a mensagem do usuário ao histórico
        chat_history.extend(contents)

        # Envia o histórico completo para manter o contexto (opcional, dependendo do modelo)
        try:
            response_chunks = client.models.generate_content_stream(
                model=model,
                contents=chat_history,
                config=generate_content_config,
            )
            print("Gemini:", end=" ")
            full_response = ""
            for chunk in response_chunks:
                # print(chunk.text, end="")
                full_response += chunk.text
            print("\n")
            # Adiciona a resposta do modelo ao histórico
            chat_history.append(types.Content(role="model", parts=[types.Part.from_text(text=full_response)]))
            print("-------------------------------------------------------------")
            print("Resposta do JarGemini( Seu assistente de Jardinagem ):")
            print("-------------------------------------------------------------")
            display(to_markdown(full_response))
            print("-------------------------------------------------------------")

        except Exception as e:
            print(f"Ocorreu um erro na comunicação com o modelo: {e}")

In [ ]:
# Executando a função do chat assistente de jardimagem
if __name__ == "__main__":
    print("Bem-vindo ao JarGemini, seu assistente de jardinagem! Digite 'sair' para encerrar.")
    chat()